```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Sketchbook to design the software for multiple drifters


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from scipy.special import lambertw

import pyopencl
import os
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble
from SWESimulators import DataAssimilationUtils as dautils


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
cl_queue = pyopencl.CommandQueue(cl_ctx)
print "Using ", cl_ctx.devices[0].name

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- drift a drifter
- applies a localized small-scale error
- observes the drifter position

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


### Utility functions

## Define a small ensemble

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 32
ny = 32

dx = 4.0
dy = 4.0

dt = 0.05*3
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 1.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)


reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
reload(dautils)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)

sim.perturbState(q0_scale=50)

ensemble_size = 4
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, cl_ctx, 
                                                 #observation_type=dautils.ObservationType.UnderlyingFlow)
                                                 observation_type=dautils.ObservationType.DirectUnderlyingFlow)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(#observation_variance_factor=2.0,
                                observation_variance = 0.01**2,
                                small_scale_perturbation_amplitude=q0)
                                #initialization_variance_factor_ocean_field=50)

ensemble.init(driftersPerOceanModel=3)


for p in range(len(ensemble.particles)):
    ensemble.particles[p].perturbState(q0_scale=30)
t = ensemble.step(dt*100)    
    
#ensemble.plotEnsemble()


In [ ]:
print "Number of particles: ", ensemble.getNumParticles()
print "Number of drifters per model: ", ensemble.driftersPerOceanModel
print "---------------------------------------------------\n"

print "true drifter position: "
print ensemble.observeTrueDrifters()
print "before change: 1D array (2)"
print "after  change: 2D array (drifters, 2)"

print " "
print "particle drifter positions:"
print ensemble.observeDrifters()
print "before change: 2D array (particles, 2)"
print "after  change: 3D array (particles, drifters, 2)"
print "---------------------------------------------------\n"

print "true state: "
print ensemble.observeTrueState()
print "before change: 1D array (4)"
print "after  change: 2D array (drifters, 4)"
print " "

print "particle states: "
print ensemble.observeParticles()
print "before change: 2D array (particles, 2)"
print "after  change: 3D array (particles, drifters, 2)"
print "---------------------------------------------------\n"

print "innovations: "
print ensemble.getInnovations()
print "before change: 2D array"
print "after  change: 3D array (particles, drifters, 2)""
print " "

print "norm of innovations: "
print ensemble.getInnovationNorms()
print "before change: 1D array"
print "after  change: 1D array (particles)"
print "---------------------------------------------------\n"


print "distances: "
print ensemble.getDistances()
print "before change: ?? array"
print "after  change: 2D array (particles, drifters)"


In [ ]:
fig = ensemble.plotDistanceInfo(title="Final ensemble")

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rmseUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.05*np.ones_like(resampling_points), 'o')
plt.title("RMSE under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.varianceUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 0.05*np.ones_like(resampling_points), 'o')
plt.title("Variance under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 0.6])

fig = plt.figure(figsize=(10,3))
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_eta, label='eta')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hu,  label='hu')
plt.plot(ensemble.tArray, ensemble.rUnderDrifter_hv,  label='hv')
plt.plot(resampling_points, 1.0*np.ones_like(resampling_points), 'o')
plt.title("r = var/rmse under drifter")
plt.legend(loc=0)
plt.grid()
plt.ylim([0, 5])

print np.sqrt(ensemble.observation_cov[0,0])

In [ ]:
def r(glob_x, cutoff):
       
    x = glob_x.copy()
    
    # map x so that glob_x is in the range 0:7
    x = x*5.0/cutoff
    
    for i in range(len(x)):
        if x[i] < 2:
            x[i] = 2
    return  np.exp(-(x-2)**2/5.0)

cutoff = 5
x = np.linspace(0, cutoff*1.5, 100)
fig = plt.figure(figsize=(6,3))
plt.plot(x, r(x, cutoff))
plt.grid()
plt.title("Cutoff: "+str(cutoff))

cutoff = 20
x = np.linspace(0, cutoff*1.5, 100)
fig = plt.figure(figsize=(6,3))
plt.plot(x, r(x, cutoff))
plt.grid()
plt.title("Cutoff: "+str(cutoff))

cutoff =10
x = np.linspace(0, cutoff*1.5, 100)
fig = plt.figure(figsize=(6,3))
plt.plot(x, r(x, cutoff))
plt.grid()
plt.title("Cutoff: "+str(cutoff))

